In [24]:
import pandas as pd
import numpy as np
from pprint import pprint
import geojson
import json

In [25]:
rki_data = pd.read_csv("../data/2021-10-07_RKI_Fallzahlen_Fixed.csv")

rki_data = rki_data[rki_data.NR.notnull()]
rki_data.loc[rki_data.LK.str.contains("Berlin"), 'LK'] = "SK Berlin"

rki_data = rki_data.groupby(['LK']).mean().reset_index()

In [28]:
rki_data

,LK,NR,LKNR,18.11.2020,19.11.2020,20.11.2020,21.11.2020,22.11.2020,23.11.2020,24.11.2020,...,28.09.21,29.09.21,30.09.21,01.10.21,02.10.21,03.10.21,04.10.21,05.10.21,06.10.21,07.10.21
0,LK Ahrweiler,2.0,7131.0,143.8,156.1,159.1,156.1,172.2,139.1,139.9,...,46.0,38.3,29.9,26.8,27.6,25.3,24.5,26.8,29.9,35.3
1,LK Aichach-Friedberg,3.0,9771.0,144.8,138.9,158.2,173.8,167.8,156.0,139.6,...,70.4,74.1,80.0,82.9,88.9,79.2,102.2,105.2,105.2,94.8
2,LK Alb-Donau-Kreis,4.0,8425.0,114.7,117.2,123.8,124.8,121.8,122.8,118.2,...,72.7,65.6,67.1,67.6,52.5,44.9,44.4,56.0,55.5,59.5
3,LK Altenburger Land,5.0,16077.0,145.4,153.3,129.8,205.8,212.5,200.2,219.3,...,43.0,47.5,43.0,43.0,55.5,66.8,79.2,74.7,86.0,89.4
4,LK Altenkirchen,6.0,7132.0,63.7,58.2,66.8,90.1,97.8,104.8,104.8,...,39.5,38.0,38.7,38.0,35.6,34.9,35.6,34.9,31.8,20.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,SK Worms,405.0,7319.0,105.3,92.2,131.7,132.9,135.3,180.7,192.7,...,135.4,107.8,125.8,113.8,95.9,98.3,112.6,70.7,76.7,68.3
397,SK Wuppertal,407.0,5124.0,189.2,193.7,176.9,203.9,198.3,198.3,194.9,...,77.2,73.0,67.3,58.3,56.3,55.8,50.7,51.5,49.9,44.5
398,SK Würzburg,408.0,9663.0,104.7,97.7,96.1,100.1,83.6,74.3,91.5,...,85.1,81.9,74.8,69.3,63.8,60.7,53.6,55.1,57.5,60.7
399,SK Zweibrücken,411.0,7320.0,93.6,99.4,90.7,96.5,73.1,46.8,64.3,...,29.4,26.5,26.5,17.6,17.6,17.6,17.6,17.6,17.6,11.8


In [41]:
with open('../data/landkreise_simplify0.geojson') as f:
    gj = geojson.load(f)
features = gj['features']

In [42]:
def updateCountyNames(geoFeature):

    county_number = float(geoFeature["properties"]["RS"])

    county_type = geoFeature["properties"]["BEZ"]
    prefix = "SK" if (county_type in ["Stadtkreis", "Kreisfreie Stadt"]) else "LK" if county_type == "Landkreis" else None
    # county_name = geoFeature["properties"]["GEN"].replace("Städte", "")
    county_name = geoFeature["properties"]["GEN"]
    if prefix:
        county_name = f"{prefix} {county_name}"


    # currently built on the assumtion that each county in the geoJSON has at most 1 match in the rki data
    rki_match = rki_data[(rki_data.LKNR == county_number) | (county_name == rki_data.LK)].to_dict(orient='records')
    if rki_match:
        geoFeature["properties"].update(rki_match[0])
    
    return geoFeature

output = list(map(updateCountyNames, features))

with open("../data/rki_geo_2.json", 'w') as outfile:
                json.dump(gj, outfile)



In [51]:

with open("../data/dates.json", 'w') as outfile:
                json.dump(list(rki_data.columns[3:]), outfile)